<a href="https://colab.research.google.com/github/GarettGazay/ai_projects/blob/master/RBSeq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time

In [144]:
rides = pd.read_csv('/content/ordering_dataset.csv')

In [145]:
rides = rides.drop(['Unnamed: 0','order','rider.first_name','rider.last_name','pickup.event_location.address1','dropoff.event_location.address1'], axis=1)

In [146]:
rides.head()

,space_type,billable_distance,run_name,rider.mediroutes_rider_id,pickup.event_location.zip,pickup.event_location.longitude,pickup.event_location.latitude,dropoff.event_location.zip,dropoff.event_location.longitude,dropoff.event_location.latitude,pickup_month,pickup_day,pickup_hour,pickup_minute,dropoff_hour,dropoff_minute
0,1.000000,0.012314,0.181818,0.184127,0.508772,-0.996364,0.992340,0.857143,-0.990424,0.848616,1.0,1.0,0.095238,0.517241,0.000000,0.818182
1,1.000000,0.006394,0.181818,0.198413,0.350877,-0.996461,0.989864,0.825397,-0.990674,0.847189,1.0,1.0,0.142857,0.517241,0.105263,0.000000
2,1.000000,0.003826,0.181818,0.182540,0.824561,-0.996339,0.990125,0.825397,-0.990674,0.847189,1.0,1.0,0.142857,0.517241,0.105263,0.000000
3,0.333333,0.023389,0.136364,0.220635,0.771930,-0.996058,0.990114,0.682540,-0.989395,0.845235,1.0,1.0,0.238095,0.206897,0.210526,0.000000
4,0.333333,0.018819,0.136364,0.092063,0.719298,-0.995385,0.991005,0.746032,-0.989807,0.845745,1.0,1.0,0.285714,0.517241,0.263158,0.545455


In [147]:
rides = rides.astype('float32')

In [148]:
rides.dtypes

space_type                          float32
billable_distance                   float32
run_name                            float32
rider.mediroutes_rider_id           float32
pickup.event_location.zip           float32
pickup.event_location.longitude     float32
pickup.event_location.latitude      float32
dropoff.event_location.zip          float32
dropoff.event_location.longitude    float32
dropoff.event_location.latitude     float32
pickup_month                        float32
pickup_day                          float32
pickup_hour                         float32
pickup_minute                       float32
dropoff_hour                        float32
dropoff_minute                      float32
dtype: object

In [149]:
rides.head()

,space_type,billable_distance,run_name,rider.mediroutes_rider_id,pickup.event_location.zip,pickup.event_location.longitude,pickup.event_location.latitude,dropoff.event_location.zip,dropoff.event_location.longitude,dropoff.event_location.latitude,pickup_month,pickup_day,pickup_hour,pickup_minute,dropoff_hour,dropoff_minute
0,1.000000,0.012314,0.181818,0.184127,0.508772,-0.996364,0.992340,0.857143,-0.990424,0.848616,1.0,1.0,0.095238,0.517241,0.000000,0.818182
1,1.000000,0.006394,0.181818,0.198413,0.350877,-0.996461,0.989864,0.825397,-0.990674,0.847189,1.0,1.0,0.142857,0.517241,0.105263,0.000000
2,1.000000,0.003826,0.181818,0.182540,0.824561,-0.996339,0.990125,0.825397,-0.990674,0.847189,1.0,1.0,0.142857,0.517241,0.105263,0.000000
3,0.333333,0.023389,0.136364,0.220635,0.771930,-0.996058,0.990114,0.682540,-0.989395,0.845235,1.0,1.0,0.238095,0.206897,0.210526,0.000000
4,0.333333,0.018819,0.136364,0.092063,0.719298,-0.995385,0.991005,0.746032,-0.989807,0.845745,1.0,1.0,0.285714,0.517241,0.263158,0.545455


In [150]:
rides['pickup_day'] = rides['pickup_day'].astype(int)
rides['pickup_month'] = rides['pickup_month'].astype(int)

rides['pickup_day'] = rides['pickup_day'].astype(str)
rides['pickup_month'] = rides['pickup_month'].astype(str)

rides['id'] = rides['pickup_month'] + rides['pickup_day']

rides = rides.drop(['pickup_month','pickup_day'], axis=1)

In [151]:
rides.head()

,space_type,billable_distance,run_name,rider.mediroutes_rider_id,pickup.event_location.zip,pickup.event_location.longitude,pickup.event_location.latitude,dropoff.event_location.zip,dropoff.event_location.longitude,dropoff.event_location.latitude,pickup_hour,pickup_minute,dropoff_hour,dropoff_minute,id
0,1.000000,0.012314,0.181818,0.184127,0.508772,-0.996364,0.992340,0.857143,-0.990424,0.848616,0.095238,0.517241,0.000000,0.818182,11
1,1.000000,0.006394,0.181818,0.198413,0.350877,-0.996461,0.989864,0.825397,-0.990674,0.847189,0.142857,0.517241,0.105263,0.000000,11
2,1.000000,0.003826,0.181818,0.182540,0.824561,-0.996339,0.990125,0.825397,-0.990674,0.847189,0.142857,0.517241,0.105263,0.000000,11
3,0.333333,0.023389,0.136364,0.220635,0.771930,-0.996058,0.990114,0.682540,-0.989395,0.845235,0.238095,0.206897,0.210526,0.000000,11
4,0.333333,0.018819,0.136364,0.092063,0.719298,-0.995385,0.991005,0.746032,-0.989807,0.845745,0.285714,0.517241,0.263158,0.545455,11


In [152]:
rides = rides.astype(str)

In [153]:
rides.values[0].astype('float32')

array([ 1.      ,  0.012314,  0.181818,  0.184127,  0.508772, -0.996364,
        0.99234 ,  0.857143, -0.990424,  0.848616,  0.095238,  0.517241,
        0.      ,  0.818182, 11.      ], dtype=float32)

Put rides into buckets by date

In [154]:
# Group the rows by the 'id' column
grouped = rides.groupby('id')

# Create a list to store the grouped arrays
day_buckets = []

# Loop through each group
for name, group in grouped:
    # Convert the group to a numpy array
    group_array = group.values.astype(np.float32)
    
    # Append the group array to the list of grouped arrays
    day_buckets.append(group_array)

# Set the display format of the numpy arrays
np.set_printoptions(precision=6, suppress=True)

# Sort the list of grouped arrays by the first column of each array
day_buckets.sort(key=lambda x: x[0][14])


In [155]:
# Do this or when you shuffle below it will shuffle both for some fucking reason, and copy doesn't work.
day_buckets_y = [x.astype(str) for x in day_buckets]

In [156]:
day_buckets_y[0][0]

array(['1.0', '0.0123136155', '0.18181819', '0.18412699', '0.50877196',
       '-0.99636376', '0.99233985', '0.85714287', '-0.9904242',
       '0.8486156', '0.0952381', '0.51724136', '0.0', '0.8181818', '11.0'],
      dtype='<U32')

In [157]:
def shuffle_np_array(np_array_list):
    for i in range(len(np_array_list)):
        np.random.shuffle(np_array_list[i])
    return np_array_list

day_buckets_X = shuffle_np_array(day_buckets)[:-1] # create a shuffled version for X below

Below each 'word' is represented by a vector in the sentence we are trying to predict the order of.

In [158]:
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the number of features in each ride vector
num_features = 15

# Define the maximum length of the sequences
max_length = 15

X = []
y = []
# Convert the input data into a 3D array of shape (num_samples, max_length, num_features)
for i in day_buckets_X:
  X.append(pad_sequences(i, maxlen=max_length, dtype='float32', padding='post'))
for i in day_buckets_y:
  y.append(pad_sequences(i, maxlen=max_length, dtype='float32', padding='post'))

In [159]:
X[0]

array([[ 0.333333,  0.023389,  0.136364,  0.220635,  0.77193 , -0.996058,
         0.990114,  0.68254 , -0.989395,  0.845235,  0.238095,  0.206897,
         0.210526,  0.      , 11.      ],
       [ 1.      ,  0.014415,  0.227273,  0.133333,  0.614035, -0.995456,
         0.991803,  0.825397, -0.990674,  0.847189,  0.52381 ,  0.206897,
         0.473684,  1.      , 11.      ],
       [ 1.      ,  0.      ,  0.363636,  0.128571,  0.719298, -0.995456,
         0.990852,  0.730159, -0.989923,  0.847515,  0.52381 ,  0.517241,
         0.526316,  0.272727, 11.      ],
       [ 1.      ,  0.006394,  0.181818,  0.198413,  0.350877, -0.996461,
         0.989864,  0.825397, -0.990674,  0.847189,  0.142857,  0.517241,
         0.105263,  0.      , 11.      ],
       [ 1.      ,  0.012314,  0.181818,  0.184127,  0.508772, -0.996364,
         0.99234 ,  0.857143, -0.990424,  0.848616,  0.095238,  0.517241,
         0.      ,  0.818182, 11.      ],
       [ 1.      ,  0.005842,  0.863636,  0.185714

In [160]:
y[0]

array([[ 1.      ,  0.012314,  0.181818,  0.184127,  0.508772, -0.996364,
         0.99234 ,  0.857143, -0.990424,  0.848616,  0.095238,  0.517241,
         0.      ,  0.818182, 11.      ],
       [ 1.      ,  0.006394,  0.181818,  0.198413,  0.350877, -0.996461,
         0.989864,  0.825397, -0.990674,  0.847189,  0.142857,  0.517241,
         0.105263,  0.      , 11.      ],
       [ 1.      ,  0.003826,  0.181818,  0.18254 ,  0.824561, -0.996339,
         0.990125,  0.825397, -0.990674,  0.847189,  0.142857,  0.517241,
         0.105263,  0.      , 11.      ],
       [ 0.333333,  0.023389,  0.136364,  0.220635,  0.77193 , -0.996058,
         0.990114,  0.68254 , -0.989395,  0.845235,  0.238095,  0.206897,
         0.210526,  0.      , 11.      ],
       [ 0.333333,  0.018819,  0.136364,  0.092063,  0.719298, -0.995385,
         0.991005,  0.746032, -0.989807,  0.845745,  0.285714,  0.517241,
         0.263158,  0.545455, 11.      ],
       [ 1.      ,  0.007237,  0.363636,  0.077778

In [161]:
input = []
target = []
for array in X:
    new_array = np.delete(array, -1, axis=1)
    input.append(new_array)

for array in y:
    new_array = np.delete(array, -1, axis=1)
    target.append(new_array)


In [162]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='mae', patience=5)

In [163]:
input[0].shape

(10, 14)

In [164]:
target[0].shape

(10, 14)

In [165]:
len(input[0])

10

In [166]:
len(target[0])

10

In [167]:
# Define the input and output shape
input_shape = (10, 14)
output_shape = (10, 14)

# input = np.random.rand(*input_shape)
# target = np.random.rand(*output_shape)


In [168]:
input[0].shape

(10, 14)

In [169]:
target[0].shape

(10, 14)

In [170]:
input = input[0]
target = target[0]

In [171]:
input = input.reshape(1, *input.shape)
target = target.reshape(1, *target.shape)

# Define the encoder
encoder_inputs = keras.layers.Input(shape=input_shape)
encoder = keras.layers.LSTM(64, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = keras.layers.Input(shape=(None, 14))
decoder_lstm = keras.layers.LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(14, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='mean_squared_error')

# Create validation data for the day


# Train the model
history = model.fit([input, target], target, epochs=100, batch_size=16, validation_data=(X_val, y_val), callbacks=[early_stopping])

ValueError: ignored

In [ ]:
# Plot history (also known as a loss curve)
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot()
plt.ylabel("loss")
plt.xlabel("epochs");

In [ ]:
pred = model.predict([input, target])

In [ ]:
pred[0][1]

In [ ]:
target[0][1]